In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import sqlite3 # library for working with sqlite database
conn = sqlite3.connect("./data/data.db") # Create a connection to the on-disk database

In [2]:
pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,patients,patients,2,"CREATE TABLE patients(ROW_ID INT,SUBJECT_ID IN..."
1,table,admissions,admissions,639,"CREATE TABLE admissions(ROW_ID INT,SUBJECT_ID ..."
2,table,icustays,icustays,3515,"CREATE TABLE icustays(ROW_ID INT,SUBJECT_ID IN..."
3,table,diagnoses_icd,diagnoses_icd,4944,"CREATE TABLE diagnoses_icd(ROW_ID INT,SUBJECT_..."
4,table,drgcodes,drgcodes,9061,"CREATE TABLE drgcodes(ROW_ID INT,SUBJECT_ID IN..."


In [3]:
pd.read_sql("""SELECT * FROM admissions LIMIT 10""", conn)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,WHITE,,,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,,,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,,PROTESTANT QUAKER,SINGLE,WHITE,,,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,,,V-TACH,0,1
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,,CATHOLIC,,WHITE,,,NEWBORN,0,1
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,WHITE,,,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,,,UNSTABLE ANGINA\CATH,0,1
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,,CATHOLIC,MARRIED,WHITE,,,STATUS EPILEPTICUS,1,1


In [4]:
pat_df = pd.read_sql("SELECT * FROM patients", conn)
adm_df = pd.read_sql("SELECT * FROM admissions", conn)
pat_df.shape[0]

46520

In [5]:
pat_df.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,,,,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,,1
2,236,251,M,2090-03-15 00:00:00,,,,0
3,237,252,M,2078-03-06 00:00:00,,,,0
4,238,253,F,2089-11-26 00:00:00,,,,0


In [6]:
all_admissions = pd.read_sql("""SELECT * FROM admissions""", conn)
print(all_admissions.shape)
all_icustays = pd.read_sql("""SELECT * from icustays""", conn)
print(all_icustays.shape)
all_icustays.head()

(58976, 19)
(61532, 12)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.249
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.06
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [7]:
# 'First' admission if it is a patient's first ever admission
# Exclude patients who die during their 'first' visit

one_admission_deaths = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
WHERE HOSPITAL_EXPIRE_FLAG = 1
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) = 1
""", conn)
print("People who died on their first admission:")
print(one_admission_deaths.shape[0])
print(one_admission_deaths.head())

one_admission_survivors = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
WHERE HOSPITAL_EXPIRE_FLAG = 0
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) = 1
ORDER BY CAST(SUBJECT_ID AS UNSIGNED) ASC
""", conn)
print("People who only had one admission but survived it:")
print(one_admission_survivors.shape[0])
print(one_admission_survivors.head())

mult_admissions = pd.read_sql("""SELECT SUBJECT_ID FROM admissions
GROUP BY SUBJECT_ID
HAVING COUNT(SUBJECT_ID) > 1
ORDER BY CAST(SUBJECT_ID AS UNSIGNED) ASC
""", conn)
print("People who had multiple admissions:")
print(mult_admissions.shape[0])
print(mult_admissions.head())

# Concatenate all subject IDs from patients who survived their first visit
patients_cols = pd.concat([one_admission_survivors, mult_admissions], axis = 0)
patients_cols.columns = patients_cols.columns.str.strip()
patients_cols = patients_cols.sort_values('SUBJECT_ID')
print("Patients who survived their first visit: " + str(patients_cols.shape[0]))
patients_cols[5:15]

People who died on their first admission:
5772
   SUBJECT_ID
0           9
1          12
2          21
3          31
4          56
People who only had one admission but survived it:
35522
   SUBJECT_ID
0           2
1           3
2           4
3           5
4           6
People who had multiple admissions:
7537
   SUBJECT_ID
0          17
1          21
2          23
3          34
4          36
Patients who survived their first visit: 43059


,SUBJECT_ID
5,7
6,8
7,10
8,11
9,13
10,16
0,17
11,18
12,19
13,20


In [8]:
# Get HADM_ID for first admission of each relevant patient


In [9]:
# Features for all patients
# Only take table info for subjects that are included in our design matrix
# Filter via inner join
admissions_info = pd.read_sql("""SELECT adm.SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME
                                 FROM admissions adm
                                 INNER JOIN patients pat
                                 ON adm.SUBJECT_ID = pat.SUBJECT_ID
                                 ORDER BY pat.SUBJECT_ID ASC
                                 """, conn)
admissions_info['DISCHTIME'] = pd.to_datetime(admissions_info['DISCHTIME'])
admissions_info['ADMITTIME'] = pd.to_datetime(admissions_info['ADMITTIME'])

all_icustays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, OUTTIME, LAST_CAREUNIT, LOS
                              FROM icustays icu
                              INNER JOIN patients pat
                              ON icu.SUBJECT_ID = pat.SUBJECT_ID
                              ORDER BY pat.SUBJECT_ID ASC
                              """, conn)

diagnoses_icd_feats = pd.read_sql("""SELECT diag.SUBJECT_ID, HADM_ID, ICD9_CODE
                                  FROM diagnoses_icd diag
                                  INNER JOIN patients pat
                                  ON diag.SUBJECT_ID = pat.SUBJECT_ID
                                  ORDER BY pat.SUBJECT_ID ASC
                                  """, conn)
# Left joined DRG codes because a lot of DRG severity/mortality data is missing
drgcodes_feats = pd.read_sql("""SELECT drg.SUBJECT_ID, HADM_ID, DRG_SEVERITY, DRG_MORTALITY 
                             FROM drgcodes drg
                             LEFT JOIN patients pat
                             ON drg.SUBJECT_ID = pat.SUBJECT_ID
                             ORDER BY pat.SUBJECT_ID ASC
                             """, conn)

In [10]:
# Further process information for admissions and ICU stays
# Calculate length of stay for each admission
admissions_feats= pd.read_sql("""SELECT adm.SUBJECT_ID, HADM_ID
                                 FROM admissions adm
                                 INNER JOIN patients pat
                                 ON adm.SUBJECT_ID = pat.SUBJECT_ID
                                 ORDER BY pat.SUBJECT_ID ASC
                                 """, conn)
# 'HOS_LOS' is length of stay in HOSPITAL, in seconds ('TimedeltaProperties' object only
# has days, seconds, microseconds as attributes)
admissions_feats['HOS_LOS'] = (admissions_info['DISCHTIME'] - admissions_info['ADMITTIME']).dt.seconds

# Only get information from LAST ICU stay from an admission
# icu_stay_counts = all_icustays['HADM_ID'].value_counts()
single_icu_stays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, LAST_CAREUNIT, LOS
                                    FROM icustays icu
                                    INNER JOIN patients pat
                                    ON icu.SUBJECT_ID = pat.SUBJECT_ID
                                    GROUP BY HADM_ID
                                    HAVING COUNT(HADM_ID) = 1
                                    ORDER BY icu.SUBJECT_ID ASC
                                    """, conn)
multiple_icu_stays = pd.read_sql("""SELECT icu.SUBJECT_ID, HADM_ID, LAST_CAREUNIT, LOS
                                    FROM icustays icu
                                    INNER JOIN patients pat
                                    ON icu.SUBJECT_ID = pat.SUBJECT_ID
                                    GROUP BY HADM_ID
                                    HAVING COUNT(HADM_ID) > 1
                                    ORDER BY icu.SUBJECT_ID ASC
                                    """, conn)
print(single_icu_stays.shape)
print(multiple_icu_stays.shape)
multiple_icu_stays.columns = multiple_icu_stays.columns.str.strip()

(54526, 4)
(3260, 4)


In [11]:
# Age
pat_adm_df = pd.read_sql("""SELECT pat.SUBJECT_ID, pat.DOB, adm.ADMITTIME
                            FROM patients pat
                            INNER JOIN admissions adm
                            ON pat.SUBJECT_ID = adm.SUBJECT_ID
                            ORDER BY pat.SUBJECT_ID ASC""", conn)

admissions_feats['age'] = (pd.to_datetime(pat_adm_df['ADMITTIME']) - pd.to_datetime(pat_adm_df['DOB'])).dt.days

In [13]:
icustays_feats = single_icu_stays
# icustays_feats2 = pd.DataFrame(columns=['SUBJECT_ID', 'HADM_ID', 'LAST_CAREUNIT', 'LOS'])

list_to_append = []
for row in range(multiple_icu_stays.shape[0]):
    if row % 200 == 0:
        print("Processing row " + str(row) + " of " + str(multiple_icu_stays.shape[0]))
    curr_hadm = multiple_icu_stays["HADM_ID"][row]
    hadm_icu_stays = pd.read_sql("""SELECT SUBJECT_ID, HADM_ID, OUTTIME, LAST_CAREUNIT, LOS
                                    FROM icustays
                                    WHERE HADM_ID = """ + str(curr_hadm), conn)
    hadm_icu_stays = hadm_icu_stays.sort_values('OUTTIME')
    temp = hadm_icu_stays[['SUBJECT_ID', 'HADM_ID', 'LAST_CAREUNIT', 'LOS']][-1:]
    icustays_feats = pd.concat([icustays_feats, temp], axis=0)
    
print("FINAL MATRIX:")
# icustays_feats = pd.DataFrame(icustays_list, columns=['SUBJECT_ID', 'HADM_ID', 'LAST_CAREUNIT', 'LOS'])
icustays_feats = icustays_feats.sort_values('SUBJECT_ID')
icustays_feats[0:10]


Processing row 0 of 3260
Processing row 200 of 3260
Processing row 400 of 3260
Processing row 600 of 3260
Processing row 800 of 3260
Processing row 1000 of 3260
Processing row 1200 of 3260
Processing row 1400 of 3260
Processing row 1600 of 3260
Processing row 1800 of 3260
Processing row 2000 of 3260
Processing row 2200 of 3260
Processing row 2400 of 3260
Processing row 2600 of 3260
Processing row 2800 of 3260
Processing row 3000 of 3260
Processing row 3200 of 3260
FINAL MATRIX:


,SUBJECT_ID,HADM_ID,LAST_CAREUNIT,LOS
0,2,163353,NICU,0.0918
1,3,145834,MICU,6.0646
2,4,185777,MICU,1.6785
3,5,178980,NICU,0.0844
4,6,107064,SICU,3.6729
1,7,118037,NICU,0.7391
5,8,159514,NICU,1.0755
6,9,150750,MICU,5.3231
7,10,184167,NICU,8.0921
8,11,194540,SICU,1.5844


In [14]:
print(admissions_info.shape)
admissions_info.head()

(58976, 4)


,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00
1,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00
2,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00
3,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00
4,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00


In [15]:
admissions_feats.head()

,SUBJECT_ID,HADM_ID,HOS_LOS,age
0,2,163353,74640,0
1,3,145834,67800,27950
2,4,185777,65580,17475
3,5,178980,27840,0
4,6,107064,31500,24084


In [16]:
print(all_icustays.shape)
all_icustays[0:10]

(61532, 5)


,SUBJECT_ID,HADM_ID,OUTTIME,LAST_CAREUNIT,LOS
0,2,163353,2138-07-17 23:32:21,NICU,0.0918
1,3,145834,2101-10-26 20:43:09,MICU,6.0646
2,4,185777,2191-03-17 16:46:31,MICU,1.6785
3,5,178980,2103-02-02 08:06:00,NICU,0.0844
4,6,107064,2175-06-03 13:39:54,SICU,3.6729
5,7,118037,2121-05-23 22:01:00,NICU,0.2677
6,7,118037,2121-05-25 21:10:19,NICU,0.7391
7,8,159514,2117-11-21 14:24:55,NICU,1.0755
8,9,150750,2149-11-14 20:52:14,MICU,5.3231
9,10,184167,2103-07-06 13:51:43,NICU,8.0921


In [17]:
print(diagnoses_icd_feats.shape)
diagnoses_icd_feats.head()

(651047, 3)


,SUBJECT_ID,HADM_ID,ICD9_CODE
0,2,163353,V3001
1,2,163353,V053
2,2,163353,V290
3,3,145834,0389
4,3,145834,78559


In [18]:
print(drgcodes_feats.shape)
drgcodes_feats[15:25]

(125557, 4)


,SUBJECT_ID,HADM_ID,DRG_SEVERITY,DRG_MORTALITY
15,16,103251,,
16,17,194023,1,1
17,17,194023,,
18,17,161087,,
19,17,161087,3,3
20,18,188822,,
21,19,109235,3,3
22,19,109235,,
23,20,157681,,
24,21,111970,,


In [19]:
# Get one overall features matrix for ALL patients
features_for_all_mat = pd.merge(admissions_feats, icustays_feats[['HADM_ID', 'LAST_CAREUNIT', 'LOS']], how='left', on=['HADM_ID'])
print(features_for_all_mat.shape)

# Merge feature dataframe with the patient_cols dataframes so we only keep the features
# for the RELEVANT patients
features_mat = pd.merge(patients_cols, features_for_all_mat, how='left', on=['SUBJECT_ID'])
print(features_mat.shape)
features_mat[0:20]

(58976, 6)
(56407, 6)


,SUBJECT_ID,HADM_ID,HOS_LOS,age,LAST_CAREUNIT,LOS
0,2,163353,74640,0,NICU,0.0918
1,3,145834,67800,27950,MICU,6.0646
2,4,185777,65580,17475,MICU,1.6785
3,5,178980,27840,0,NICU,0.0844
4,6,107064,31500,24084,SICU,3.6729
5,7,118037,75120,0,NICU,0.7391
6,8,159514,14280,0,NICU,1.0755
7,10,184167,2040,0,NICU,8.0921
8,11,194540,45720,18316,SICU,1.5844
9,13,143045,73920,14560,CSRU,3.666


In [20]:
# Generate additional columns:
# Append truth labels column (Was patient readmitted within 30 days?)
# Append 'ICU stay?' column
subj_adm = admissions_feats["SUBJECT_ID"].value_counts()
subj_adm = subj_adm.sort_index()

mult_adm_subj = subj_adm.index[subj_adm > 1]

# Instantiate to all zeros
truth = np.zeros((len(subj_adm)))

subj_adm_idx = list(subj_adm.index)

for subject in mult_adm_subj:
    temp_df = admissions_info[admissions_info["SUBJECT_ID"] == subject]
    # Get time between first discharge and second admission
    first_disc_time = temp_df["DISCHTIME"].iloc[0]
    second_adm_time = temp_df["ADMITTIME"].iloc[1]
    days_to_readm = (pd.to_datetime(second_adm_time) - pd.to_datetime(first_disc_time)).days
    
    # If readmitted within 30 days, set truth to 1
    if days_to_readm <= 30:
        truth[subj_adm_idx.index(subject)] = 1
        
truth_df = pd.DataFrame({'SUBJECT_ID': subj_adm_idx, 'READM': np.array(truth, dtype='int')})
print(truth_df.head())

   SUBJECT_ID  READM
0           2      0
1           3      0
2           4      0
3           5      0
4           6      0


In [21]:
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn import metrics
from sklearn.model_selection import train_test_split

design_mat = features_mat.merge(truth_df,how='inner',on="SUBJECT_ID")
X = design_mat.iloc[:,2:-1]

LCU = design_mat["LAST_CAREUNIT"]
le = LabelEncoder() # categorical encoder
LCU = le.fit_transform(LCU.astype(str))
X['LAST_CAREUNIT'] = LCU
X['LOS'] = pd.to_numeric(X['LOS']).fillna(0)

# normalize feature values
for i in list(X):
    X[i] = normalize(X[i][:].values.reshape(-1,1),axis=0)
    
# make train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, design_mat.iloc[:,-1],test_size = 0.33, random_state = 0)

X

,HOS_LOS,age,LAST_CAREUNIT,LOS
0,0.005955,0.000000,0.004436,0.000035
1,0.005409,0.003932,0.002957,0.002334
2,0.005232,0.002459,0.002957,0.000646
3,0.002221,0.000000,0.004436,0.000032
4,0.002513,0.003388,0.005914,0.001414
5,0.005993,0.000000,0.004436,0.000284
6,0.001139,0.000000,0.004436,0.000414
7,0.000163,0.000000,0.004436,0.003115
8,0.003648,0.002577,0.005914,0.000610
9,0.005897,0.002048,0.001479,0.001411


### Classifiers

### Metrics for evaluating models

In [22]:
# provide these metrics
# y_pred = # this is the output prediction values for the classifier
# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
# AUC = metrics.auc(fpr, tpr)
# AUC